In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from interval import interval, inf, imath



input_data = pd.read_excel('/Users/dylan/DataBase/signal_process/50ETF-signalout copy.xlsx')
signal_output = pd.read_excel('/Users/dylan/DataBase/signal_process/signal_result-30 copy.xlsx')

signal_output_quantile = signal_output[['annual_return_sp','win_rate_sp','pc_ratio_sp']].quantile([0.25,0.5,0.75])


signal_output_weight1 = signal_output_quantile[['annual_return_sp','win_rate_sp','pc_ratio_sp']].loc[0.5].loc['annual_return_sp'] < signal_output['annual_return_sp']
signal_output['annual_return_sp_weight'] = signal_output_weight1.replace({True : 1, False : 0})

signal_output_weight2 = signal_output_quantile[['annual_return_sp','win_rate_sp','pc_ratio_sp']].loc[0.5].loc['win_rate_sp'] < signal_output['win_rate_sp']
signal_output['win_rate_sp_weight'] = signal_output_weight2.replace({True : 1, False : 0})

""" signal_output_weight3 = signal_output_quantile[['annual_return_sp','win_rate_sp','pc_ratio_sp']].loc[0.5].loc['pc_ratio_sp'] < signal_output['pc_ratio_sp'] 
signal_output['pc_ratio_sp_weight'] = signal_output_weight3.replace({True : 1, False : 0}) """

interval3 = interval(signal_output_quantile[['annual_return_sp','win_rate_sp','pc_ratio_sp']].loc[0.5].loc['pc_ratio_sp'],signal_output_quantile[['annual_return_sp','win_rate_sp','pc_ratio_sp']].loc[0.75].loc['pc_ratio_sp'])

signal_output_weight3 = signal_output_quantile[signal_output['win_rate_sp'] in interval3]


signal_output['Max_weight'] = signal_output[['annual_return_sp_weight','win_rate_sp_weight','pc_ratio_sp_weight']].sum(axis = 1)

signal_output.sort_values(by = 'Max_weight' ,axis = 0 , ascending = False, inplace = True) # 注意

# 选出Top1 



 


KeyError: True

In [4]:
df = pd.DataFrame([[0, 1, -2, -1], [1, 1, 1, 1]])
df

,0,1,2,3
0,0,1,-2,-1
1,1,1,1,1


In [105]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


input_data = pd.read_excel('/Users/dylan/DataBase/signal_process/50ETF-signalout copy.xlsx')
signal_input = pd.read_excel('/Users/dylan/DataBase/signal_process/signal_result-30 copy.xlsx')

# 加权选择

signal_input_quantile = signal_input[['annual_return_sp','win_rate_sp','pc_ratio_sp']].quantile([0.25,0.5,0.75])
for col_name in ['annual_return_sp','win_rate_sp','pc_ratio_sp']:
    def get_weight_group(x) :
        if x < signal_input_quantile[col_name].loc[0.25]: 
            return 1
        elif signal_input_quantile[col_name].loc[0.25] < x  and  x < signal_input_quantile[col_name].loc[0.5]: 
            return 2 
        elif signal_input_quantile[col_name].loc[0.5] < x and  x < signal_input_quantile[col_name].loc[0.75] : 
            return 3         
        else : 
            return 4 
    new_weighte_col_name = col_name + '_weight'
    signal_input[new_weighte_col_name] = signal_input[col_name].map(get_weight_group)

signal_input['Max_method'] = signal_input[['annual_return_sp_weight','win_rate_sp_weight','pc_ratio_sp_weight']].sum(axis = 1)

signal_input['Max_method'].sort_values(ascending = False)

# 至少选择当期Top3
signal_input.sort_values(by = 'Max_method', ascending = False ).groupby('start_date').head(3)



"""  #当期相同则计算上期method
if len(signal_input_filter['Max_method']) > 3:       
    signal_input_filter.sort_values(by=['model','start_date'],inplace=True) 
    signal_input_filter['Last_method'] = signal_input_filter['Max_method'].shift(1) 
    signal_output = signal_input_filter.nlargest(3,'Last_method',keep="first")
else:
    signal_output = signal_input_filter.sort_values(by="start_date").shift(1)

    signal_input_filter.sort_values(by='start_date',inplace=True)
    signal_input_filter['model_selected'] = signal_input_filter['model'].shift(1)
    
    signal_input_filter.to_excel('/Users/dylan/DataBase/signal_process/1.xlsx',index=False) """
# 选出当期Top3

signal_input_filter = signal_input.groupby(by = 'start_date').apply(lambda x : x.nlargest(3, 'Max_method', keep = 'all'))


""" # 权重相同则比较上一期
for i in ['start_date']:
    if len(signal_input) """
signal_output = pd.DataFrame()
for i in signal_input_filter['start_date'].drop_duplicates(): 
    column = signal_input_filter.loc[(i)].sample(1)
    signal_output = signal_output.append(column)

signal_output['model_selected'] = signal_output['model'].shift(1)

### 得到信号

        
df1 = signal_output[['annual_return_sp','win_rate_sp','pc_ratio_sp']]
df3 = pd.DataFrame()
for x in signal_output['model']:
    df2 = signal_output[(x == signal_input['model'])]
    for y in signal_output['start_date']: 
        new_column = df2[y == df2['start_date']]
        df3 = df3.append(new_column)

df4 = df3.drop_duplicates('start_date')
df4.rename(columns = {'annual_return_sp_weight' : 'a','win_rate_sp_weight' : 'b','pc_ratio_sp_weight' : 'c'}, inplace = True)
df1.rename(columns = {'annual_return_sp' : 'a','win_rate_sp' : 'b','pc_ratio_sp' : 'c'}, inplace = True)
signal_output['signal'] = df1.dot(df4[['a','b','c']].T)

<ipython-input-105-64557d0dcfb5>:68: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = signal_output[(x == signal_input['model'])]


ValueError: matrices are not aligned

In [114]:
df4[['a','b','c']].T

,1624,1059,630,103,966,1281,1984,1559,1022,519,...,1254,166,1319,266,18,653,2156,1127,460,939
a,4,2,2,4,4,4,1,2,4,4,...,3,4,3,3,3,4,1,3,4,4
b,3,4,4,4,4,3,3,4,4,4,...,4,4,4,4,4,3,4,4,4,4
c,2,3,4,4,3,3,4,2,3,4,...,3,4,4,4,4,4,1,3,4,3


In [115]:
df1

,a,b,c
1624,6.02,0.50,1.14
1059,0.47,0.71,1.64
103,7.13,0.38,3.84
1281,6.81,0.55,1.41
1559,-1.83,0.70,1.19
519,11.03,0.57,2.15
630,-0.30,0.70,2.05
53,3.85,0.50,4.74
268,4.02,0.60,2.60
954,3.96,0.67,1.73


In [118]:
signal_output['signal'] = df1.dot(df4[['a','b','c']].T).T.iloc[0]
signal_output['signal']

In [122]:
Max = signal_output['signal'].max()
Min = signal_output['signal'].min()
signal_output['signal'] = (signal_output['signal'] - Min)/(Max - Min)

1624    0.325255
1059    0.197109
103     0.384313
1281    0.349240
1559    0.134002
519     0.463992
630     0.182843
53      0.316035
268     0.295477
954     0.284451
347     0.279280
1460    0.259656
184     0.235360
1394    0.206641
1698    0.177859
511     0.333790
258     0.397770
376     0.247010
1022    0.264328
1119    0.296661
168     0.339085
1547    0.196486
1183    0.198106
26      0.449975
1984    0.097558
566     0.293733
267     0.355781
1161    0.226078
1254    0.226202
431     0.538001
166     0.295103
1       1.000000
1319    0.209943
1550    0.209008
567     0.282208
266     0.232058
18      0.532083
653     0.470533
2156    0.000000
1127    0.215113
460     0.432906
939     0.405744
909     0.220346
966     0.349302
577     0.265699
Name: signal, dtype: float64